# XAFS using Larch from Python 3.6

This is an example published originally by [TitanWolf](https://titanwolf.org/Network/Articles/Article?AID=f8639528-1e63-435c-bcd4-e67ed966719a#gsc.tab=0).

Some adaptations were required to run the code are listed as follows.
- Replacing the input file for FeS2.
- using demeter generated file for input to feff6.
- generating the FEFF paths using larch feff6. 
- used fp instead of temp as temp is not defined.
- small sintax corrections for python 3.6 

## Libraries

The required libraries are shown below

In [ ]:
from larch import Interpreter
import larch_plugins as lp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

session = Interpreter()
%matplotlib inline

## Processing of Data
The following code performs the following tasks
- read the data
- background removal
- EXAFS derivation of extraction and radial structure function of vibration
- plot data

**Note** instead of the Cu file used in the example, we are using the Fe files for the demeter example.

In [ ]:
# loading data
Fe_expt = lp.io.read_ascii("fes2_rt01_mar02.xmu", labels="energy mu")#, larch=session)
# calc exafs spectrum
lp.xafs.autobk(Fe_expt, kmin=0.0, kmax=16.5, rbkg=1.0, group=Fe_expt, _larch=session)
# fourier transformation
lp.xafs.xftf(Fe_expt, kweight=2, kmin=3, kmax=15.0, window="hanning", dk=1.0, group=Fe_expt, _larch=session)

# Plot
fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
ax1.plot(Fe_expt.k, Fe_expt.k**2 * Fe_expt.chi)
ax1.set_xlim(0, Fe_expt.k.max())
ax1.set_xlabel("Wavenumber of Photoelectron/$\mathrm{\AA}^{-1}$")
ax1.set_ylabel("$k^2 \\times$ EXAFS oscillation $\chi (k)$/$\mathrm{\AA}^{-2}$")
ax2 = fig.add_subplot(122)
ax2.plot(Fe_expt.r, Fe_expt.chir_mag)
ax2.set_xlim(0, 6)
ax2.set_xlabel("Radial distance/$\mathrm{\AA}$")
ax2.set_ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")
plt.show()
 

## FEFF input file

 The feff input file fes2_feff.inp was generated from the FeS2.inp file using demeter.
 The generation of feff paths is not presented in the example

In [ ]:
# run feff and get the paths
from larch.xafs.feffrunner import feff6l
#feff6l(folder='.', feffinp='feff.inp', verbose=True)
feff6l(folder = './fes2_feff',feffinp='fes2_feff.inp' )

## Reading of the scattering path

Load an scattering path calculated in FEFF (using lp.xafs.FeffPathGroup) to read one fo the feffNNNN.dat files. 

In [ ]:
fp = lp.xafs.FeffPathGroup(filename='./fes2_feff/feff0001.dat', _larch=session)

## Calculating EXAFS vibration

### Calculate chi of sigma square
Use the \_calc_chi to  calculate vibration. The argument of this method are the parameters of the FeffPathGroup attribute table. For now, calculate sigma2 only (0.005).

k, chi, and chi_mag, value are the outputs of running \_calc_chi. 
k represents the photoelectron wave numbers, because the chi is the EXAFS spectrum. It will plot of the EXAFS vibration of chi against k.

In [ ]:
temp = fp._calc_chi(sigma2=0.005)
plt.plot(fp.k, fp.chi*fp.k**2)
plt.xlabel("Wavenumber of Photoelectron/$\mathrm{\AA}^{-1}$")
plt.ylabel("$k^2 \\times$ EXAFS oscillation $\chi (k)$/$\mathrm{\AA}^{-2}$")
plt.show()

Next, calculate the radial structure function of this path. 

To do this, use the lp.xafs.xftf as well as the experimental results. However, lp.xafs.xftf outputs the calculation result to a Group instance, so you must create a Group instance to receive the results.

In [ ]:
# make Group instance for outputs
from larch.symboltable import Group
fp_group = Group()
# calc EXAFS spectrum
fp._calc_chi(sigma2=0.005)
# fourier transform
lp.xafs.xftf(fp.k, fp.chi, kweight=2, kmin=3, kmax=15, dk=1.0, window='hanning', group=fp_group, _larch=session)
# plot
plt.plot(fp_group.r, fp_group.chir_mag)
plt.xlim(0, 6)
plt.xlabel("Radial distance/$\mathrm{\AA}$")
plt.ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")
plt.show()

The \_calc_chi method is very convenient, for example, it can be used to verify how EXAFS vibration changes when changing the parameters (sigma2, deltar, s02). 

For instance, the following code tracks the changes in  EXAFS vibration and radial structure function when varying the value of sigma square.

When the sigma square increases, EXAFS vibration is found to decay. Also, you can see that the attenuation as k is larger.

In [ ]:
sigma2 = np.arange(0.001, 0.005, 0.0005)

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for i, s in enumerate(sigma2):
    fp._calc_chi(deltar=0, s02=1, sigma2=s)
    fp_group = Group()
    lp.xafs.xftf(fp.k, fp.chi, kweight=2, kmin=3, kmax=15, dk=1.0, window='hanning', group=fp_group, _larch=session)
    ax1.plot(fp.k, fp.chi*fp.k**2, color=cm.jet(float(i) / len(sigma2)))
    ax1.set_xlabel("Wavenumber of Photoelectron/$\mathrm{\AA}^{-1}$")
    ax1.set_ylabel("$k^2 \\times$ EXAFS oscillation $\chi (k)$/$\mathrm{\AA}^{-2}$")
    ax2.plot(fp_group.r, fp_group.chir_mag, color=cm.jet(float(i) / len(sigma2)))
    ax2.set_xlim(0, 6)
    ax2.set_xlabel("Radial distance/$\mathrm{\AA}$")
    ax2.set_ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")
plt.show()
 

When deltar changes, you can see that the oscillation period is changed. The amplitude is also noticeable, the peak position of the radial structure function is the Fourier transform shifts.

In [ ]:
delr = np.arange(0.0, 0.1, 0.025)

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)


for i, r in enumerate(delr):
    fp._calc_chi(deltar=r, s02=1, sigma2=0.001)
    fp_group = Group()
    lp.xafs.xftf(fp.k, fp.chi, kweight=2, kmin=3, kmax=15, dk=1.0, window='hanning', group=fp_group, _larch=session)
    ax1.plot(fp.k, fp.chi*fp.k**2, color=cm.jet(float(i) / len(delr)))
    ax1.set_xlabel("Wavenumber of Photoelectron/$\mathrm{\AA}^{-1}$")
    ax1.set_ylabel("$k^2 \\times$ EXAFS oscillation $\chi (k)$/$\mathrm{\AA}^{-2}$")
    ax2.plot(fp_group.r, fp_group.chir_mag, color=cm.jet(float(i) / len(delr)))
    ax2.set_xlim(0, 6)
    ax2.set_xlabel("Radial distance/$\mathrm{\AA}$")
    ax2.set_ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")

plt.show()

The s02 parameter acts on the amplitude in a similar manner to sigma2. However, unlike sigma2, there is no k dependence on the way of decay.

In [ ]:
s02 = np.arange(0.5, 1.0, 0.05)

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)


for i, s in enumerate(s02):
    fp._calc_chi(deltar=0.0, s02=s, sigma2=0.001)
    fp_group = Group()
    lp.xafs.xftf(fp.k, fp.chi, kweight=2, kmin=3, kmax=15, dk=1.0, window='hanning', group=fp_group, _larch=session)
    ax1.plot(fp.k, fp.chi*fp.k**2, color=cm.jet(float(i) / len(s02)))
    ax1.set_xlabel("Wavenumber of Photoelectron/$\mathrm{\AA}^{-1}$")
    ax1.set_ylabel("$k^2 \\times$ EXAFS oscillation $\chi (k)$/$\mathrm{\AA}^{-2}$")
    ax2.plot(fp_group.r, fp_group.chir_mag, color=cm.jet(float(i) / len(s02)))
    ax2.set_xlim(0, 6)
    ax2.set_xlabel("Radial distance/$\mathrm{\AA}$")
    ax2.set_ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")

plt.show()
 

When e0 is varied, the vibration period in reasponds in a a similar way to delta R.

In [ ]:
e0 = np.arange(-10.0, 10.0, 2.5)

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)


for i, e in enumerate(e0):
    fp._calc_chi(deltar=0.0, s02=1.0, sigma2=0.001, e0=e)
    fp_group = Group()
    lp.xafs.xftf(fp.k, fp.chi, kweight=2, kmin=3, kmax=15, dk=1.0, window='hanning', group=fp_group, _larch=session)
    ax1.plot(fp.k, fp.chi*fp.k**2, color=cm.jet(float(i) / len(e0)))
    ax1.set_xlabel("Wavenumber of Photoelectron/$\mathrm{\AA}^{-1}$")
    ax1.set_ylabel("$k^2 \\times$ EXAFS oscillation $\chi (k)$/$\mathrm{\AA}^{-2}$")
    ax2.plot(fp_group.r, fp_group.chir_mag, color=cm.jet(float(i) / len(e0)))
    ax2.set_xlim(0, 6)
    ax2.set_xlabel("Radial distance/$\mathrm{\AA}$")
    ax2.set_ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")
 
plt.show()

## Synthesis of path
In this example, the FEFF calculation produced 48 path, wich are stored as 48 feffNNNN.dat files. 
The paths can be read toguether for processing. 
The following code creates a list of paths.

In [ ]:
import glob
dat_list = glob.glob('.\\fes2_feff\\feff*.dat')
paths = []

for i, f in enumerate(dat_list):
    i = i + 1
    #print(f)
    num_string = str(i).zfill(4)
    # this will create 48 new variables named path00NN where NN ranges from 01 to 48.
    # Each is added to the list of paths
    exec('path'+num_string+' = lp.xafs.FeffPathGroup(filename=f, _larch=session)')
    exec('path'+num_string+'._calc_chi(sigma2=0.004)')
    exec('paths.append(path'+num_string+')')
 

In [ ]:
vars(paths[0])

After this, it is possible to plot the sum of the scattered paths contained in the list.

In [ ]:
all_path_sum = np.zeros(paths[0].k.shape)
k_calc = paths[0].k
for p in paths:
    all_path_sum += p.chi

all_path_group = Group()

lp.xafs.xftf(k_calc, all_path_sum, kweight=2, kmin=3, kmax=15, dk=1.0, window='hanning', group=all_path_group, _larch=session)

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.plot(k_calc+0.1, all_path_sum*fp.k**2, label='sim. (sum of all paths)') # used fp instead of temp
ax1.plot(Fe_expt.k, Fe_expt.chi*Fe_expt.k**2, label='expt.')
ax1.set_xlim(0, 16)
ax1.set_xlabel("Wavenumber of Photoelectron/$\mathrm{\AA}^{-1}$")
ax1.set_ylabel("$k^2 \\times$ EXAFS oscillation $\chi (k)$/$\mathrm{\AA}^{-2}$")
ax1.legend()
ax2.plot(all_path_group.r, all_path_group.chir_mag, label='sim. (sum of all paths)')
ax2.plot(Fe_expt.r, Fe_expt.chir_mag, label='expt.')
ax2.set_xlim(0, 6)
ax2.set_xlabel("Radial distance/$\mathrm{\AA}$")
ax2.set_ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")
ax2.legend()
plt.show()
 

In [ ]:
print ("Direct print of path0001:", path0001.nleg)
print ("Print of path0001 in the paths list:", paths[0].nleg)

Given that nleg is 2 for single scattering paths, thus the value of nleg can be used to select paths. 

The follwoing example adds up only single scattering paths.

In [ ]:
ss_path_sum = np.zeros(paths[0].k.shape)
k_calc = paths[0].k

for p in paths:
    if p.nleg == 2:
        ss_path_sum += p.chi
 

Afther this, the sum of single scattering paths can be compared to the sum of all scattering paths and plotted.

In [ ]:
ss_path_group = Group()
lp.xafs.xftf(k_calc, ss_path_sum, kweight=2, kmin=3, kmax=15, dk=1.0, window='hanning', group=ss_path_group, _larch=session)

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.plot(k_calc+0.1, all_path_sum*fp.k**2, label='sum of all paths') # used fp instead of temp
ax1.plot(k_calc+0.1, ss_path_sum*fp.k**2, label='sum of single scattering paths') # used fp instead of temp
ax1.set_xlim(0, 16)
ax1.set_xlabel("Wavenumber of Photoelectron/$\mathrm{\AA}^{-1}$")
ax1.set_ylabel("$k^2 \\times$ EXAFS oscillation $\chi (k)$/$\mathrm{\AA}^{-2}$")
ax1.legend()
ax2.plot(all_path_group.r, all_path_group.chir_mag, label='sum of all paths')
ax2.plot(ss_path_group.r, ss_path_group.chir_mag, label='sum of single scattering paths')
ax2.set_xlim(0, 6)
ax2.set_xlabel("Radial distance/$\mathrm{\AA}$")
ax2.set_ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")
ax2.legend(loc='upper right')
plt.show()
 

## Fitting of the EXAFS spectrum

The Fitting pocess of the EXAFS spectrum using Larch is described as follows.

- Create a group to define the fitting parameters.
- Read an scattering path calculated in FEFF.
- Create a lp.xafs.TransformGroup.
- Create a lp.xafs.FeffitDataSet.
- Carry out the fitting in lp.xafs.feffit.

The following code creates the parameters group.
The code is changed to use lp.fitting.param_group and lp.fitting.param instead of importing  Group and Parameter (does not work).

In [ ]:
pars = lp.fitting.param_group(amp    = lp.fitting.param(1.0, vary=True),
                   del_e0 = lp.fitting.param(0.0, vary=True),
                   sig2   = lp.fitting.param(0.0, vary=True),
                   del_r  = lp.fitting.param(0.0, vary=True), _larch=session )

The following code loads the scattering path and assigns the parameters defined previously to s02, e0, sigma2, and deltar (amp, del_e0, sig2, and del_r respectively).

In [ ]:
path1 = lp.xafs.FeffPathGroup(filename = 'fes2_feff/feff0001.dat',
                              s02      = 'amp',
                              e0       = 'del_e0',
                              sigma2   = 'sig2',
                              deltar   = 'del_r',
                              _larch=session)

XAFS of fitting performs the following operations:
- Fitting of the radial structure function (R space)
- Fitting the inverse Fourier transform over a specific range of the radial structure function (Q space)
- EXAFS is fitting the vibration (k-space)

The measured data and simulation results (the Fourier transform, etc.) need to be transformed to make them comparable.
The lp.xafs.TransformGroup is used to prepare the fitspace.

In [ ]:
trans = lp.xafs.TransformGroup(fitspace='r', kmin=3, kmax=14, kw=2, dk=1, window='hanning', rmin=1.4,
                               rmax=3.0, _larch=session)

Next a set object is created to hold the experimental results, the model and the above TransformGroup, an instance of  lp.xafs.FeffitDataSet. 

Spectra, etc. of the fitting results are also stored in this FeffitDataSet.

In [ ]:
dataset = lp.xafs.FeffitDataSet(data=Fe_expt, pathlist=[path1], transform=trans, _larch=session)

Finally, run the fitting using the lp.xafs.feffit. The arguments for the firt are the parameters Group and the  FeffitDataSet.

The fit result after executing (lp.xafs.feffit) is  in the variable named out. 

In [ ]:
out = lp.xafs.feffit(pars, dset, _larch=session)

Additionally, EXAFS vibration and radial structure are stored in the FeffitDataSet instance specified at the time of execution of lp.xafs.feffit. these can be plotted.

In [ ]:
fig = plt.figure()
plt.plot(dset.data.r, dset.data.chir_mag, color='b')
plt.plot(dset.data.r, dset.data.chir_re, color='b', label='expt.')
plt.plot(dset.model.r, dset.model.chir_mag, color='r')
plt.plot(dset.model.r, dset.model.chir_re, color='r', label='fit')
plt.ylabel("Magnitude of Fourier Transform of $k^2 \cdot \chi$/$\mathrm{\AA}^{-3}$")
plt.xlabel("Radial distance/$\mathrm{\AA}$")
plt.xlim(0, 5)
plt.ylim(-8, 8)
plt.fill([1.4, 1.4, 3.0, 3.0],[-8, 8, 8, -8], color='g',alpha=0.1)
plt.text(2.35, -7.5, 'fit range')
plt.legend()
plt.show()

The results of the fit can be printed using feffit_report.

In [ ]:
print(lp.xafs.feffit_report(out, _larch=session))